# Worksheet 05

Name: Hrishav Varma

UID: U57996211


### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

First calculating distance of each point from each centroid:

### Iteration 1:

- Point 0:    (0, 2)      -> Assigned Cluster 1
- Point 0.5:  (0.5, 1.5)  -> Assigned Cluster 1
- Point 1.5:  (1.5, 0.5)  -> Assigned Cluster 2
- Point 2:    (2, 0)      -> Assigned Cluster 2
- Point 6:    (6, 4)      -> Assigned Cluster 2
- Point 6.5:  (6.5, 4.5)  -> Assigned Cluster 2
- Point 7:    (7, 5)      -> Assigned Cluster 2

With the assigned points, calculating new centroids:
- Cluster 1: New Centroid (0, 0.5)/2          = 0.25
- Cluster 2: New Centroid (1.5+2+6+6.5+7)/5   = 4.6


### Iteration 2:

- Point 0:    (0.25, 4.6) -> Assigned Cluster 1
- Point 0.5:  (0.25, 4.1) -> Assigned Cluster 1
- Point 1.5:  (1.25, 3.1) -> Assigned Cluster 1
- Point 2:    (1.75, 2.6) -> Assigned Cluster 1
- Point 6:    (5.75, 1.4) -> Assigned Cluster 2
- Point 6.5:  (6.25, 1.9) -> Assigned Cluster 2
- Point 7:    (6.75, 2.4) -> Assigned Cluster 2

With the assigned points, calculating new centroids:
- Cluster 1: New Centroid (0+0.5+1.5+2)/4 = 1
- Cluster 2: New Centroid (6+6.5+7)/3 = 6.5

### Iteration 3:

- Point 0:    (1, 6.5)    -> Assigned Cluster 1
- Point 0.5:  (0.5, 6)    -> Assigned Cluster 1
- Point 1.5:  (0.5, 5)    -> Assigned Cluster 1
- Point 2:    (1, 4.5)    -> Assigned Cluster 1
- Point 6:    (5.5, 0.5)  -> Assigned Cluster 2
- Point 6.5:  (5.5, 0)    -> Assigned Cluster 2
- Point 7:    (6, 0.5)    -> Assigned Cluster 2

Since the points did not change what cluster they belonged to, we have found the clustering we were looking for using Lloyd's algorithm.

- Cluster 1: Centroid (1), Points: (0, 0.5, 1.5, 2)
- Cluster 2: Centroid (6.5), Points: (6, 6.5, 7)

b) Describe in plain english what the cost function for k means is.

The cost function for K-means finds the total squared distance between each data point and its cluster centroid. This distance is used as the function to minimize which then defines which cluster centroid the data point belongs to. This function is used to find the optimized clustering of the given data points.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

For the same number of clusters K, there could be very different solutions to the k means algorithm on a given dataset because it would depend on the what initial centroids are taken as this could mean the algorithm converges to different local minimas of the cost function and not necessarily the global minima of the cost function. Similarly it could depend be upon the distribution of the data points aswell.

d) Does Lloyd's Algorithm always converge? Why / why not?

Lloyd's algorithm always converges becaus it keeps decreasing the value of objective function but it may only converge to a local minimum and not necessarily always converge to a global minima.

e) Follow along in class the implementation of Kmeans

In [1]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[ 1.20175677  1.1086792 ]
 [-3.52713792  1.62040555]
 [-1.72164425  1.93561193]
 [ 1.80582124 -4.04080901]]
[[ 1.31365209  1.15532655]
 [-3.64201483  1.76599375]
 [-1.61784051  1.4953298 ]
 [ 1.80582124 -4.04080901]]
[[ 1.52736903  1.33654705]
 [-3.66600698  1.81128047]
 [-1.4028886   1.04487039]
 [ 1.80582124 -4.04080901]]
[[ 1.60238653  1.48000404]
 [-3.56347063  1.85617202]
 [-1.2179936   0.77382237]
 [ 1.82416454 -4.00058334]]
[[ 1.71138506  1.5991525 ]
 [-3.32282706  1.90401737]
 [-0.84220978  0.4159547 ]
 [ 1.82416454 -4.00058334]]
[[ 1.8391211   1.79153538]
 [-3.18218195  1.90395221]
 [-0.44322287  0.16160269]
 [ 1.84180743 -4.03258775]]
[[ 1.90165131  1.93255625]
 [-3.12164037  1.85868514]
 [-0.18461573  0.10805748]
 [ 1.84180743 -4.03258775]]
[[ 1.904406    1.98450725]
 [-3.07703402  1.85266958]
 [-0.03760357  0.01175879]
 [ 1.84585117 -4.10098602]]
[[ 1.91248297  2.00204407]
 [-3.07703402  1.85266958]
 [-0.01886062  0.02039989]
 [ 1.84585117 -4.10098602]]
[[ 1.91248297  2.00